In [1]:
using Distributed, Folds, StaticArrays
using Random, Distributions

In [2]:
include("perm_test.jl")
include("partition.jl")

partition (generic function with 1 method)

In [11]:
n, n1, n2 = 200, 12, 8
d = Normal()
Random.seed!(123)
x1s = rand(d, (n, n1))
x2s = rand(d, (n, n2))
data= hcat(x1s, x2s)

200×20 Matrix{Float64}:
 -0.645731   -0.605691   -0.756464   …   0.037911   -0.609532    0.268983
 -1.46325    -0.354813    0.21732       -1.05255    -0.236771   -1.57179
 -1.6236     -0.782707   -0.737335      -0.137629    0.33269    -0.171054
 -0.217665    0.0812944  -1.98671        0.0656706   0.526011   -0.562248
  0.492246   -0.453337    0.572599       0.406255    0.185543    0.251055
  0.98098     0.0983998  -0.979219   …  -0.736835    0.135897   -0.22668
  0.0799568   0.77467     1.72272       -0.561219   -0.357161   -0.65838
  1.54912    -2.08215    -1.37706        0.467577   -0.782272    0.304469
 -1.34161    -0.0387583  -1.3131         0.0614698   0.506474    2.50715
  0.412162   -0.701248   -0.0490801     -2.12121    -1.38048    -1.56417
  0.593197    1.30727    -0.521194   …   0.543751   -1.26496     0.494784
 -0.768409   -0.458261   -0.330495       0.842038   -0.120105    1.00935
 -0.0761679   0.232625   -0.319317      -0.52795    -0.583295    0.0196977
  ⋮                

In [5]:
@time permInterval(data[1,:], n1, n2)

  0.223484 seconds (757.29 k allocations: 579.545 MiB, 10.46% gc time)


(-1.6742101306465704, -0.07659743689226098)

In [13]:
@time Folds.map(x -> permInterval(x, n1, n2), eachrow(data))

 33.156998 seconds (151.93 M allocations: 113.338 GiB, 13.69% gc time, 0.56% compilation time)


200-element Vector{Tuple{Float64, Float64}}:
 (-0.7119425908237351, 1.0449379754369408)
 (-1.239892721941454, 0.877891176529052)
 (-2.646654613975966, -0.28878749368484624)
 (-0.9244068880302111, 0.9439692814449955)
 (-1.9229711017115767, 0.36593589761435646)
 (-1.3565902567673858, 0.12471002139126643)
 (-0.6065665850305321, 1.3691445577419779)
 (-2.0695105117765102, 0.1398563851993619)
 (-1.6651021902976053, 0.5432687464094414)
 (-1.1139823096069927, 1.2241170130412318)
 (-1.4279152667367034, 0.45972748242876726)
 (-1.1342673492096398, 0.4513095755020735)
 (-0.07247101998126507, 1.2278874947197966)
 ⋮
 (-0.8446542337846246, 0.8980332589134414)
 (-0.8844527156228399, 0.8896442911382032)
 (-0.852614455427628, 1.3198019836974726)
 (-1.4602156673065196, 0.7565565142253674)
 (-0.4626942194357558, 1.9158250362492835)
 (-1.2700264532797871, 0.5772477899156723)
 (-2.011356389512537, 0.40320556889565057)
 (-1.3534129053095985, 0.9162690069635281)
 (-1.606790373543701, 0.557816391827329)
 (-1.3

In [10]:
@time pmap(x -> permInterval(x, n1, n2), eachrow(data), batch_size=2)

 20.623253 seconds (75.82 M allocations: 56.721 GiB, 13.06% gc time, 0.37% compilation time)


100-element Vector{Tuple{Float64, Float64}}:
 (-1.6742101306465704, -0.07659743689226098)
 (-1.7144016375850728, -0.30352235980414266)
 (-2.3513784637891906, 0.580579151260358)
 (-0.5255073075358477, 1.0270406957714533)
 (-2.030259691926244, 0.889541097686495)
 (-1.5130095682831557, 0.36955805333649183)
 (-0.7259367382877664, 1.2416716162723098)
 (-1.2111533635333016, 1.4002141108356554)
 (-1.4868625857262323, 0.9018686701242511)
 (-1.9801065161013673, -0.04688670035144976)
 (-1.3388640668948413, 0.7145593329791826)
 (-1.2584043479202718, 0.20043942189843217)
 (-1.3314467008176398, 0.42804282833382)
 ⋮
 (-1.6530679527250005, 0.4394081104592495)
 (-2.2018680542076763, 0.40534380232791006)
 (-0.9721386456991785, 0.9336264560248604)
 (-0.7211325874244914, 1.203799134750064)
 (-1.9333789460264232, 0.5194446441055253)
 (-0.7655258851661386, 1.8142259024106253)
 (-0.9215857206465485, 1.0305077785183143)
 (-0.4321139860292218, 1.4797230160026782)
 (0.03527877557121643, 1.300863384070392)
 (-0

In [12]:
# ~400 intervals/min
intervals = repeat([(0.0,0.0)], n)
@time begin
    Threads.@threads for i in 1:n
        intervals[i] = permInterval(data[i,:], n1, n2)
    end
end

 31.755264 seconds (151.48 M allocations: 113.314 GiB, 13.35% gc time, 0.09% compilation time)


In [45]:
intervals[1:5]

5-element Vector{Tuple{Float64, Float64}}:
 (-1.6742101306465704, -0.07659743689226098)
 (-1.7144016375850728, -0.30352235980414266)
 (-2.3513784637891906, 0.580579151260358)
 (-0.5255073075358477, 1.0270406957714533)
 (-2.030259691926244, 0.889541097686495)